In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model, ensemble, tree, neural_network, svm
import sklearn.model_selection
from preprocessing import *
import xgboost as xgb
from xgboost.sklearn import XGBClassifier  
from public_accuracy import score_function
from combine_classifiers import *

In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, SimpleRNN, GRU

/home/etienne/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/etienne/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
X_0 = pd.read_csv('./data/train_engineered.csv', index_col=0, skiprows=[1], nrows=None)
Y_0 = pd.read_csv('./data/train_label_clean.csv', sep=",", index_col=0, nrows=None)

In [4]:
cols = X_0.columns.tolist()
multi_index = [c.split('.')[0] for c in cols]
multi_index = [(multi_index[i], i%1440) for i in range(len(multi_index))]
stats = [multi_index[1440*i][0] for i in range(len(multi_index)//1440)]
print(stats)

['assist', 'bad pass', 'block', 'defensive rebound', 'lost ball', 'miss', 'offensive foul', 'offensive rebound', 'score', 'steals', 'FG_guest', 'FG_home', 'off_reb_effi_guest', 'off_reb_effi_home', 'average_poss_guest', 'average_poss_home']


In [5]:
X_0 = np.swapaxes(X_0.values.reshape(X_0.shape[0], -1, 1440), 1, 2)
y = Y_0.values

## Train XGBoost

In [6]:
# sampling rate in seconds
sampling1 = 120
X = subsampling(X_0, sampling1).reshape(X_0.shape[0], -1)

In [7]:
print(X.shape)

(12499, 192)


In [8]:
# cross validation
ncross = 4
train_acc = []
valid_acc = []

group_kfold = sklearn.model_selection.KFold(n_splits=ncross, shuffle=True)
group_kfold.get_n_splits(X)

for train_index, valid_index in group_kfold.split(X):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]
    classifier = XGBClassifier(max_depth=10, eta=0.1, silent=0, objective='binary:logistic', gamma=0, n_estimators=100,
                               min_child_weight=0, max_delta_step=0, subsampling=1, reg_lambda=1, booster='dart',
                               rate_drop=0.2)
    classifier.fit(X_train, y_train)
    
    train_acc.append(classifier.score(X_train, y_train))
    valid_acc.append(classifier.score(X_valid, y_valid))
    
print('Accuracy on training set: ', np.mean(train_acc))
print('Accuracy on validation set: ', np.mean(valid_acc))

/home/etienne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/etienne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/etienne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/etienne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Retu

Accuracy on training set:  0.9653303833297775
Accuracy on validation set:  0.7271779257362356


/home/etienne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/etienne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
del X, X_train, y_train, X_valid, y_valid

## Train RandomForest

In [10]:
# sampling rate in seconds
sampling2 = 120
X = subsampling(X_0, sampling2).reshape(X_0.shape[0], -1)

In [11]:
print(X.shape)

(12499, 192)


In [12]:
# cross validation
ncross = 4
train_acc = []
valid_acc = []

group_kfold = sklearn.model_selection.KFold(n_splits=ncross, shuffle=True)
group_kfold.get_n_splits(X)

for train_index, valid_index in group_kfold.split(X):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]
    classifier = ensemble.RandomForestClassifier(n_estimators=200, criterion='entropy',
                                                 max_features="auto", max_depth=None)
    classifier.fit(X_train, y_train)
    
    train_acc.append(classifier.score(X_train, y_train))
    valid_acc.append(classifier.score(X_valid, y_valid))
    
print('Accuracy on training set: ', np.mean(train_acc))
print('Accuracy on validation set: ', np.mean(valid_acc))

/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Accuracy on training set:  1.0
Accuracy on validation set:  0.7397399487836107


In [13]:
del X, X_train, y_train, X_valid, y_valid

## Train RNN

In [14]:
def RNN (batch_size, epochs, input_dim, input_length):
    
    model = Sequential()
    # we add a Convolution1D, which will learn filters
    # word group filters of size filter_length:
    model.add(SimpleRNN(30,
                     activation='relu',
                     input_dim=input_dim,
                     input_length=input_length))
    model.add(Dropout(0.2))

    # We add a vanilla hidden layer:
    model.add(Dense(25))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    model.add(Dense(25))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))



    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

In [15]:
# sampling rate in seconds
sampling3 = 10
X = subsampling(X_0, sampling3)

In [16]:
print(X.shape)

(12499, 144, 16)


In [17]:
batch_size = 32
epochs = 10

# cross validation
ncross = 4
train_acc = []
valid_acc = []

group_kfold = sklearn.model_selection.KFold(n_splits=ncross, shuffle=True)
group_kfold.get_n_splits(X)

for train_index, valid_index in group_kfold.split(X):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    model = RNN(batch_size, epochs, input_dim=len(stats), input_length=1440//sampling3)

    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=False,
              validation_data=(X_valid, y_valid))

    tacc = np.mean(y_train.reshape(-1, 1) == model.predict_classes(X_train, verbose=False))
    vacc = np.mean(y_valid.reshape(-1, 1) == model.predict_classes(X_valid, verbose=False))
    train_acc.append(tacc)
    valid_acc.append(vacc)
    
print('Accuracy on training set: ', np.mean(train_acc))
print('Accuracy on validation set: ', np.mean(valid_acc))

/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  if __name__ == '__main__':
/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(30, activation="relu", input_shape=(144, 16))`
  if __name__ == '__main__':


Accuracy on training set:  0.7240578792404524
Accuracy on validation set:  0.7202975928297055


## Combine classifiers

In [18]:
X_train, X_valid = X_0[train_index], X_0[valid_index]
y_train, y_valid = y[train_index], y[valid_index]

In [19]:
print(X_train.shape)

(9375, 1440, 16)


In [20]:
model1 = XGBClassifier(max_depth=10, eta=0.1, silent=0, objective='binary:logistic', gamma=0, n_estimators=100,
                               min_child_weight=0, max_delta_step=0, subsampling=1, reg_lambda=1, booster='dart',
                               rate_drop=0.2)

In [21]:
model2 = ensemble.RandomForestClassifier(n_estimators=200, criterion='entropy',
                                                 max_features="auto", max_depth=None)

In [22]:
batch_size = 32
epochs = 10
model3 = RNN(batch_size, epochs, input_dim=len(stats), input_length=1440//sampling3)

/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  if __name__ == '__main__':
/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(30, activation="relu", input_shape=(144, 16))`
  if __name__ == '__main__':


In [25]:
cclass = CombineClassifiers([(model1, 'XGBoost'), (model2, 'RandomForest'), (model3, 'RNN')], nsamples=[sampling1, sampling2, sampling3], combine='regression')
cclass.fit(X_train, y_train, valid_ratio=0.1, batch_size=32, epochs=10)       
acc = np.mean(cclass.predict(X_valid)==y_valid)
train_acc = np.mean(cclass.predict(X_train)==y_train)
print('Accuracy on train:', train_acc)
print('Accuracy on validation:', acc)

/home/etienne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/etienne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


704/938 [=====================>........] - ETA: 0s

/home/etienne/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


9088/9375 [============================>.] - ETA: 0sAccuracy on train: 0.95008
Accuracy on validation: 0.7371959026888605


In [26]:
cclass.combine = 'median'
acc = np.mean(cclass.predict(X_valid)==y_valid)
train_acc = np.mean(cclass.predict(X_train)==y_train)
print('Accuracy on train:', train_acc)
print('Accuracy on validation:', acc)

9344/9375 [============================>.] - ETA: 0sAccuracy on train: 0.9376
Accuracy on validation: 0.734314980793854


In [27]:
cclass.combine = 'mean'
acc = np.mean(cclass.predict(X_valid)==y_valid)
train_acc = np.mean(cclass.predict(X_train)==y_train)
print('Accuracy on train:', train_acc)
print('Accuracy on validation:', acc)

9184/9375 [============================>.] - ETA: 0sAccuracy on train: 0.9336533333333333
Accuracy on validation: 0.7285531370038413


# Submit a classifier

In [28]:
classifier = ensemble.RandomForestClassifier(n_estimators=200, criterion='entropy',
                                                 max_features="auto", max_depth=None)
sampling = 60
X = subsampling(X_0, sampling).reshape(X_0.shape[0], -1)
classifier.fit(X, y)

/home/etienne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [29]:
test = pd.read_csv('./data/test_engineered.csv', index_col=0, skiprows=[1], nrows=10)

In [30]:
X_test = np.swapaxes(test.values.reshape(test.shape[0], -1, 1440), 1, 2)
X_test = subsampling(X_test, sampling).reshape(X_test.shape[0], -1)

In [31]:
print(X_test.shape)

(10, 384)


In [32]:
y_test = classifier.predict(X_test)

In [35]:
test_label = pd.DataFrame(y_test, index=test.index, columns=['label'])

In [36]:
test_label.to_csv('tests/predictions.csv', sep=';')